![](https://www.nuplan.org/static/media/nuPlan_final.3fde7586.png)

### Contents

1. [Introduction to nuPlan](#introduction)
2. [Training an ML planner](#training)
3. [Simulating a planner](#simulation)
4. [Visualizing metrics and scenarios](#dashboard)

In [1]:
from tutorials.utils.tutorial_utils import setup_notebook

setup_notebook()

# Introduction to nuPlan <a name="introduction"></a>


Welcome to nuPlan! This notebook will explore the nuPlan simulation framework, training platform as well as the nuBoard metrics/scenarios visualization dashboard.

## What is nuPlan

nuPlan is the world’s first closed-loop ML-based planning benchmark for autonomous driving.

It provides a high quality dataset with 1500h of human driving data from 4 cities across the US and Asia with widely varying traffic patterns (Boston, Pittsburgh, Las Vegas and Singapore). In addition, it provides a closed-loop simulation framework with reactive agents, a training platform as well as a large set of both general and scenario-specific planning metrics.

![](https://www.nuscenes.org/static/media/framework_steps.2d4642df.png)

## Training & simulation framework

The nuPlan training and simulation framework aims to:
* create a simulation pipeline to evaluate a planner on large dataset with various scenarios
* score planner performance with common and scenario-dependent metrics
* compare planners based on measured metrics and provide intuitive visualizations
* train planners with the provided framework to allow quick implementation and iteration
* support closed-loop simulation and training

<br />

![](https://www.nuplan.org/static/media/planning_framework.ca3c2969.png)

## Scenarios in nuPlan

nuPlan aims to capture challenging yet representative scenarios from real-world encounters. This enables the benchmarking of planning systems both in expert imitation (open-loop) and reactive planning (closed-loop) settings.

These scenarios includes:
* highly interactive scenes with traffic participants (e.g. tailgating, high-velocity overtakes, double parked cars, jaywalking)
* various ego behaviors (e.g. vehicle following, yielding, lane merging) and dynamics (e.g. mixed speed profiles, abrupt braking, speed bumps, high jerk maneuvers)
* scene layouts of varied complexity (e.g. pudos, traffic/stop controlled intersections, unprotected turns) and temporary zones (e.g. construction areas)

The dataset is automatically tagged with scenario labels based on certain primitive attributes.
These scenario tags can then be used to extract representative metrics for the planner's evaluation.

Example mined scenarios in nuPlan:

| | | |
| :-: | :-: | :-: |
| Unprotected cross turn | Dense vehicle interactions | Jaywalker in front |
| ![](https://www.nuscenes.org/static/media/unprotected-cross.51feef7e.webp) | ![](https://www.nuscenes.org/static/media/dense-interactions.16de47ec.webp) | ![](https://www.nuscenes.org/static/media/jaywalker.03083823.webp) |
| Lane change | Ego at pickup/dropoff area | Ego following vehicle |
| ![](https://www.nuscenes.org/static/media/lane-change.54bfca1c.webp) | ![](https://www.nuscenes.org/static/media/pickup-dropoff.4dd1c418.webp) | ![](https://www.nuscenes.org/static/media/following-vehicle.4cacd559.webp) |

## Database

Download a database for training/simulation from [here](https://nuplan.org/nuplan#download).

| Database | Size | Duration | Num Logs | Cities | Num Scenarios | Sensor Data | Description |
| :-- | :-- | :-- | :-- | :-- | :-- | :-- | :-- |
| nuplan_v1.1_mini (recommended) | 13GB | 7h | 64 | Las Vegas, Boston, Pittsburgh, Singapore | 67 | N/A | The mini split used for prototyping and testing |
| nuplan_v1.1 | 1.8TB | 1282h | 15910 | Las Vegas, Boston, Pittsburgh, Singapore | 73 | N/A | The full dataset for training and evaluation|

## Setup

To be able to access all resources within this notebook, make sure Jupyter is launched at the root of this repo. The path of the notebook should be `/notebook/<repo_root>`.

In [2]:
# (Optional) Increase notebook width for all embedded cells to display properly
from IPython.core.display import display, HTML
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_3053484/4180358289.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
# Useful imports
import os
from pathlib import Path
import tempfile

import hydra

# Training an ML planner <a name="training"></a>

## Imitation learning
In the following section we will train an ML planning policy with the aim estimate the ego's future trajectory and control the vehicle.

The policy is learned through imitation learning, a supervised learning approach in which - in the context of autonomous driving - the behavior of an expert human driver is used as a target signal to supervise the model.

## Model features & targets
A planning policy consumes a set of episodic observations and encodes them through a deep neural network to regress a future trajectory.

The observations can be historic or present ego and agent poses as well as static/dynamic map information across different map layers.<br />
These signals can be encoded through various representations, such as raster or vector format for the map signal, each with their pros and cons for each model flavor.

Using these input features the model predicts a discretized future trajectory across a fixed time horizon.<br />
The trajectory consists of a set of discrete future states (position, heading and velocity) sampled at fixed intervals which express the likelihood of the vehicle being at that state in the future.<br />
For example, a predicted trajectory may consist of 10 future poses sampled at intervals of 0.5s across a 5s horizon.

## Learning objectives
The policy is trained to maximize a set of aggregated objectives such as imitation, collision avoidance, traffic rule violation etc.<br />
Imitation is the core training objective which indicates how close the predicted trajectory is to the expert ground truth and penalizes model predictions that deviate in space and time from the demonstration.

## Training parameters

The following parameter categories define the training protocol which includes the model, metrics, objectives etc.

A working example composition of these parameters can be found in the next section.

---

### ML models

Change the training model with `model=X` where `X` is a config yaml defined in the table below. 

| Model | Description | Config |
| --- | --- | --- |
| Raster model (CNN) | Raster-based model that uses a CNN backbone to encode ego, agent and map information as raster layers<br />Any (pretrained) backbone from the TIMM library can be used (e.g. ResNet50, EfficientNetB3) | `raster_model` |
| Vector model (LaneGCN) | Vector-based model that uses a series of MLPs to encode ego and agent signals, a lane graph to encode vector-map elements and a fusion network to capture lane & agent intra/inter-interactions through attention layers<br />Implementation of LaneGCN paper ("Learning Lane Graph Representations for Motion Forecasting") | `vector_model` |
| Simple vector model | Toy vector-based model that consumes ego, agent and lane signals through a series of MLPs | `simple_vector_model` |

<br />

### Training objectives

Change the training objectives with `objective=[X, ...]` where `X` is a config yaml defined in the table below. 

| Objective | Description | Config |
| --- | --- | --- |
| Imitation objective | Penalizes the predicted trajectory that deviates from the expert demonstration | `imitation_objective` |

<br />

### Training metrics

Change the training objectives with `training_metric=[X, ...]` where `X` is a config yaml defined in the table below. 

| Metric | Description | Config |
| --- | --- | --- |
| Average displacement error | RMSE translation error across full predicted trajectory | `avg_displacement_error` |
| Average heading error | RMSE heading error across full predicted trajectory | `avg_heading_error` |
| Final displacement error | L2 error of predicted trajectory's final pose translation | `final_displacement_error` |
| Final heading error | L2 error of predicted trajectory's final pose heading | `final_heading_error` |

## Prepare the training config

In [ ]:
import os
NUPLAN_DATA_ROOT = os.getenv('NUPLAN_DATA_ROOT', '/mnt/slurmfs-A100/user_data/ryao092/datasets/nuplan/')
NUPLAN_MAPS_ROOT = os.getenv('NUPLAN_MAPS_ROOT', '/mnt/slurmfs-A100/user_data/ryao092/datasets/nuplan/maps')
NUPLAN_DB_FILES = os.getenv('NUPLAN_DB_FILES', '/mnt/slurmfs-A100/user_data/ryao092/datasets/nuplan/nuplan-v1.1/splits/mini')
NUPLAN_MAP_VERSION = os.getenv('NUPLAN_MAP_VERSION', 'nuplan-maps-v1.0')
os.environ['NUPLAN_DATA_ROOT'] = NUPLAN_DATA_ROOT
os.environ['NUPLAN_MAPS_ROOT'] = NUPLAN_MAPS_ROOT
os.environ['NUPLAN_DB_FILES'] = NUPLAN_DB_FILES
os.environ['NUPLAN_MAP_VERSION'] = NUPLAN_MAP_VERSION

# Location of path with all training configs
CONFIG_PATH = '../nuplan/planning/script/config/training'
CONFIG_NAME = 'default_training'

# Create a temporary directory to store the cache and experiment artifacts
SAVE_DIR = Path(tempfile.gettempdir()) / 'tutorial_nuplan_framework'  # optionally replace with persistent dir
# 定义实验名称
EXPERIMENT = 'training_raster_experiment'
# 定义任务名称，实验配置
JOB_NAME = 'train_default_raster'
# 定义日志输出目录
LOG_DIR = SAVE_DIR / EXPERIMENT / JOB_NAME

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # 清除之前的hydra实例
hydra.initialize(config_path=CONFIG_PATH)               # 使用指定的配置路径初始化hydra

# Compose the configuration 组合配置参数
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'group={str(SAVE_DIR)}',                               # 设置实验组目录 
    f'cache.cache_path={str(SAVE_DIR)}/cache',              # 设置缓存路径
    f'experiment_name={EXPERIMENT}',                        # 设置实验名称
    f'job_name={JOB_NAME}',                                 # 设置任务名称
    'py_func=train',                                        # 指定要执行的Python函数为训练函数
    '+training=training_raster_model',  # raster model that consumes ego, agents and map raster layers and regresses the ego's trajectory # 使用栅格模型进行训练，该模型使用ego、agents和地图栅格层并回归ego的轨迹
    'scenario_builder=nuplan_mini',  # use nuplan mini database                      # 使用nuplan mini数据库
    'scenario_filter.limit_total_scenarios=500',  # Choose 500 scenarios to train with 选择500个场景进行训练
    'lightning.trainer.params.accelerator=ddp_spawn',  # ddp is not allowed in interactive environment, using ddp_spawn instead - this can bottleneck the data pipeline, it is recommended to run training outside the notebook
    # 在交互式环境中不允许使用ddp，改用ddp_spawn（可能会成为数据管道瓶颈，建议在notebook外运行训练）
    'lightning.trainer.params.max_epochs=10',          # 设置最大训练轮数为10
    'data_loader.params.batch_size=8',                  # 设置批处理大小为8
    'data_loader.params.num_workers=8',                 # 设置数据加载器的工作进程数为8
])


## Launch tensorboard for visualizing training artifacts

In [5]:
# %load_ext tensorboard
# %tensorboard --logdir {str(LOG_DIR)} --port 60587
import socket

def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        s.listen(1)
        port = s.getsockname()[1]
    return port

free_port = find_free_port()
print(f"Using port: {free_port}")

%load_ext tensorboard
%tensorboard --logdir {str(LOG_DIR)} --port {free_port}

Using port: 56757


Launching TensorBoard...

## Launch training (within the notebook)

In [ ]:
from nuplan.planning.script.run_training import main as main_train


# Run the training loop, optionally inspect training artifacts through tensorboard (above cell)
main_train(cfg)

Global seed set to 0


2025-05-30 15:06:52,459 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:20}  Building experiment folders...
2025-05-30 15:06:52,465 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:22}  Experimental folder: /tmp/tutorial_nuplan_framework/training_raster_experiment/train_default_raster/2025.05.30.15.06.52
2025-05-30 15:06:52,465 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2025-05-30 15:06:52,506 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/utils/multithreading/worker_ray.py:78}  Starting ray local!


2025-05-30 15:06:55,273	INFO worker.py:1888 -- Started a local Ray instance.


2025-05-30 15:06:56,683 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: RayDistributed
2025-05-30 15:06:56,685 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 128
Number of GPUs per node: 4
Number of threads across all nodes: 128
2025-05-30 15:06:56,685 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2025-05-30 15:06:56,688 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/training/experiments/training.py:41}  Building training engine...
2025-05-30 15:06:56,688 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/scrip

Ray objects: 100%|██████████| 64/64 [00:03<00:00, 16.61it/s]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


2025-05-30 15:07:02,805 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/script/builders/scenario_builder.py:171}  Extracted 500 scenarios for training
2025-05-30 15:07:02,806 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/training/experiments/training.py:52}  Updating configs based on ddp_spawn strategy is currently not supported. Optimizer and LR Scheduler configs will not be updated.
2025-05-30 15:07:02,810 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/script/builders/training_callback_builder.py:19}  Building callbacks...
2025-05-30 15:07:02,818 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/nuplan-devkit/nuplan/planning/script/builders/training_callback_builder.py:37}  Building callbacks...DONE!
2025-05-30 15:07:02,824 INFO {/mnt/slurmfs-4090node1/homes/rzhong151/rzhong151/Diffusion-Planner/

Traceback (most recent call last):
  File "/mnt/slurmfs-4090node1/homes/rzhong151/anaconda3/envs/diffusion_planner/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/mnt/slurmfs-4090node1/homes/rzhong151/anaconda3/envs/diffusion_planner/lib/python3.9/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/mnt/slurmfs-4090node1/homes/rzhong151/anaconda3/envs/diffusion_planner/lib/python3.9/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/mnt/slurmfs-4090node1/homes/rzhong151/anaconda3/envs/diffusion_planner/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/mnt/slurmfs-4090node1/homes/rzhong151/anaconda3/envs/diffusion_planner/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self

In [ ]:
import pytorch_lightning as pl
print(f"PyTorch Lightning version: {pl.__version__}")

## Launch training (command line - alternative)

A training experiment with the above same parameters can be launched alternatively with:
```
$ python nuplan/planning/script/run_training.py \
    experiment_name=raster_experiment \
    py_func=train \
    +training=training_raster_model \
    scenario_builder=nuplan_mini \
    scenario_filter.limit_total_scenarios=500 \
    lightning.trainer.params.max_epochs=10 \
    data_loader.params.batch_size=8 \
    data_loader.params.num_workers=8
```

# Simulating a planner <a name="simulation"></a>

## Open-loop simulation
Open-loop simulation aims to evaluate the policy's capabilities to imitate the expert driver's behavior.<br />
This is essentially done through *log replay* as the policy's predictions do not affect the state of the simulation.

As the policy is not in full control of the vehicle, this type of simulation can only provide a high-level performance overview.

## Closed-loop simulation
Conversely, in closed-loop simulation the policy's actions alter the state of the simulation which tries to closely approximate the real-world system.

The simulation's feedback loop enables a more in-depth evaluation of the policy as compounding errors can cause future observations to significantly diverge from the ground truth.<br />
This is important in measuring distribution shifts introduced due to lack of variance in training examples through pure imitation learning.

Closed-loop simulation is further divided into two categories:
* ego closed-loop simulation with agents replayed from log (open-loop, non reactive)
* ego closed-loop simulation with agents controlled by a rule-based or learned policy (closed-loop, reactive)

## Measuring success
Measuring the success of a planning task and comparing various planning policies is a complicated effort that involves defining metrics across different vertical dimensions and scenario categories.<br />
These metrics include indicators such as vehicle dynamics, traffic rule violations, expert imitation, navigation success etc.<br />
Overall, they aim to capture the policy's ability to control the autonomous vehicle safely yet efficiently without compromising the passenger's comfort.

成功评估
评估规划任务的成功并比较各种规划策略是一项复杂的工作，涉及在不同垂直维度和场景类别中定义指标。<br /> 这些指标包括车辆动力学、交通规则违反、专家模仿、导航成功等指示器。<br /> 总体而言，它们旨在捕获策略在不影响乘客舒适度的前提下，安全且高效地控制自动驾驶车辆的能力。

## Simulation parameters

### Planners

Change the planner model with `planner=X` where `X` is a config yaml defined in the table below. 

| Planner | Description | Config |
| --- | --- | --- |
| Simple Planner | Naive planner that only plans a straight path | `simple_planner` |
| ML Planner | Learning-based planner trained using the nuPlan training framework (see previous section) | `ml_planner` |

## Prepare the simulation config

In [ ]:
# Location of path with all simulation configs  # 设置仿真配置文件的路径
CONFIG_PATH = '../nuplan/planning/script/config/simulation'
CONFIG_NAME = 'default_simulation' # 设置默认仿真配置文件名称

# Select the planner and simulation challenge # 选择规划器和仿真挑战类型
PLANNER = 'simple_planner'  # [simple_planner, ml_planner] 规划器选择
CHALLENGE = 'open_loop_boxes'  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents]
# 仿真挑战类型：开环仿真、闭环非反应式智能体、闭环反应式智能体

# 数据集参数配置
DATASET_PARAMS = [
    'scenario_builder=nuplan_mini',  # use nuplan mini database # 使用nuplan mini数据库
    'scenario_filter=all_scenarios',  # initially select all scenarios in the database # 初始选择数据库中的所有场景
    'scenario_filter.scenario_types=[near_multiple_vehicles, on_pickup_dropoff, starting_unprotected_cross_turn, high_magnitude_jerk]',  # select scenario types
    # 选择特定的场景类型：靠近多车辆、接送点、无保护左转、高冲击力
    'scenario_filter.num_scenarios_per_type=10',  # use 10 scenarios per scenario type # 每种场景类型使用10个场景
]

# Name of the experiment # 实验名称
EXPERIMENT = 'simulation_simple_experiment'

# Initialize configuration management system 
# 初始化配置管理系统
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized # 如果之前已初始化，则重新初始化hydra
hydra.initialize(config_path=CONFIG_PATH)  # 使用指定的配置路径初始化hydra

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'experiment_name={EXPERIMENT}',  # 设置实验名称
    f'group={SAVE_DIR}',               # 设置保存目录组
    f'planner={PLANNER}',               # 设置使用的规划器
    f'+simulation={CHALLENGE}',        # 设置仿真挑战类型（+号表示添加配置）
    *DATASET_PARAMS,                  # 解包数据集参数列表，包含场景构建器和过滤器设置
])

## Launch simulation (within the notebook)

In [ ]:
# 这段代码是用于运行nuPlan仿真并保存结果的
# 导入nuPlan仿真脚本的主函数
from nuplan.planning.script.run_simulation import main as main_simulation

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
# 运行仿真循环（目前还不支持实时可视化，可视化请参见下一节）
main_simulation(cfg)

# Simple simulation folder for visualization in nuBoard
# 保存简单仿真文件夹路径，用于后续在nuBoard中进行可视化
simple_simulation_folder = cfg.output_dir

## Launch simulation (command line - alternative)

A simulation experiment can be launched alternatively with:
```
$ python nuplan/planning/script/run_simulation.py \
    +simulation=open_loop_boxes \
    planner=simple_planner \
    scenario_builder=nuplan_mini \
    scenario_filter=all_scenarios \
    scenario_filter.scenario_types="[near_multiple_vehicles, on_pickup_dropoff, starting_unprotected_cross_turn, high_magnitude_jerk]" \
    scenario_filter.num_scenarios_per_type=10 \
```

## Simulate a trained ML planner for comparison

Using the same simulation settings as before, we can simulate a pretrained ML planner and compare the two.

In this example you can take the model you trained earlier.

In [ ]:
# Location of path with all simulation configs # 设置仿真配置文件的路径
CONFIG_PATH = '../nuplan/planning/script/config/simulation'
# 设置默认仿真配置文件名称
CONFIG_NAME = 'default_simulation'

# Get the checkpoint of the trained model
last_experiment = sorted(os.listdir(LOG_DIR))[-1]              # 获取最后一个实验目录
train_experiment_dir = sorted(LOG_DIR.iterdir())[-1]           # 获取最新的训练实验目录
checkpoint = sorted((train_experiment_dir / 'checkpoints').iterdir())[-1]  # 获取最新的模型检查点文件

# 处理模型路径，转义等号字符以适应Hydra配置格式
MODEL_PATH = str(checkpoint).replace("=", "\=")

# Name of the experiment # 定义ML规划器仿真实验名称
EXPERIMENT = 'simulation_raster_experiment'

# Initialize configuration management system # 初始化配置管理系统
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized # 如果之前已初始化，则重新初始化hydra
hydra.initialize(config_path=CONFIG_PATH)                                                  # 使用指定的配置路径初始化hydra

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'experiment_name={EXPERIMENT}',  # 设置实验名称
    f'group={SAVE_DIR}',   # 设置保存目录组
    'planner=ml_planner',   # 使用机器学习规划器
    'model=raster_model',  # 使用栅格模型
    'planner.ml_planner.model_config=${model}',  # hydra notation to select model config    # 使用Hydra语法选择模型配置
    f'planner.ml_planner.checkpoint_path={MODEL_PATH}',  # this path can be replaced by the checkpoint of the model trained in the previous section
    # 设置模型检查点路径（可替换为前面训练的模型检查点）
    f'+simulation={CHALLENGE}', # 设置仿真挑战类型（+号表示添加配置）
    *DATASET_PARAMS,  # 解包数据集参数列表，包含场景构建器和过滤器设置
])

# Run the simulation loop # 运行仿真循环
main_simulation(cfg)

# ML_planner simulation folder for visualization in nuBoard (next section)
# 保存ML规划器仿真文件夹路径，用于下一节的nuBoard可视化
ml_planner_simulation_folder = cfg.output_dir

# Visualizing metrics and scenarios <a name="dashboard"></a>

## nuBoard summary

Having trained and simulated planners across various scenarios and driving behaviors, it's time to evaluate them:
* quantitatively, through common and scenario dependent metrics
* qualitatively, through visualization of scenario progression

nuBoard 概述
在对各种场景和驾驶行为的规划器进行训练和仿真后，现在是时候对它们进行评估了：

定量评估：通过通用和场景相关的指标
定性评估：通过场景进展的可视化

### nuBoard tabs
To achieve that, nuBoard has 3 core evaluation tabs:
1. Overview - Scalar metrics summary of common and scenario metrics across the following categories:
    * Ego dynamics
    * Traffic violations
    * Expert imitation
    * Planning & navigation
    * Scenario performance
2. Histograms - Histograms over metric statistics for more a granular peek inside each metric focusing on:
    * Metric statistics (e.g. min, max, p90)
3. Scenarios - Low-level scenario visualizations:
    * Time-series progression of a specific metric across a scenario
    * Top-down visualization of the scenario across time for comparing predicted vs. expert trajectories

In addition, there is a main configuration tab for selecting different simulation files for comparing planners/experiments.


nuBoard 选项卡
为了实现这一目标，nuBoard 有3个核心评估选项卡：

概览 - 以下类别的通用和场景指标的标量指标摘要：
* 自车动力学
* 交通违规
* 专家模仿
* 规划与导航
* 场景表现
* 直方图 - 指标统计的直方图，用于更细粒度地查看每个指标，重点关注：
* 指标统计（例如最小值、最大值、P90分位数）
* 场景 - 低级别场景可视化：
* 特定指标在场景中的时间序列进展
* 场景随时间变化的俯视图可视化，用于比较预测轨迹与专家轨迹
* 此外，还有一个主配置选项卡，用于选择不同的仿真文件来比较规划器/实验。

<br />

**NOTE**: nuBoard is under heavy developement, overall functionality and aesthetics do not represent the final product!

注意：nuBoard 正在大力开发中，整体功能和美观性并不代表最终产品！

## Prepare the nuBoard config

In [ ]:
# Location of path with all nuBoard configs
CONFIG_PATH = '../nuplan/planning/script/config/nuboard'
CONFIG_NAME = 'default_nuboard'

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    'scenario_builder=nuplan_mini',  # set the database (same as simulation) used to fetch data for visualization
    f'simulation_path={[simple_simulation_folder, ml_planner_simulation_folder]}',  # nuboard file path(s), if left empty the user can open the file inside nuBoard
])

## Launch nuBoard (open in new tab - recommended)

In [ ]:
from nuplan.planning.script.run_nuboard import main as main_nuboard

# Run nuBoard
main_nuboard(cfg)

## Launch nuBoard (embedded within the notebook - alternative)

In [ ]:
from bokeh.io import show, output_notebook
from nuplan.planning.script.run_nuboard import initialize_nuboard

# Make sure that the notebook working directory is "/notebooks" and that Jupyter was launched at the root of the repo
cfg.resource_prefix = '/notebooks/nuplan/planning/metrics/board/'  # pass CSS resources to the notebook

# Run the nuBoard
output_notebook()
nuboard = initialize_nuboard(cfg)
show(nuboard.main_page)

## Launch nuBoard (command line - alternative)

nuBoard can be launched alternatively with:
```
$ python nuplan/planning/script/run_nuboard.py
```

Simulation files (.nuboard) can be selected under the configuration tab.